# Capstone Project - Part 2: Adding Latitude and Longitude to the data
## Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

### I was unable to retrieve the data using GeoCoder (the while loop hit the daily limit) so I've loaded the CSV file provided

In [63]:
lat_lng_coords = None
count=1

while(lat_lng_coords is None):
    g = geocoder.google('M7A, Toronto, Ontario')
    lat_lng_coords = g.latlng
    print("None", count)
    count += 1

None 1
None 2
None 3
None 4
None 5
None 6
None 7
None 8
None 9
None 10
None 11
None 12
None 13
None 14
None 15
None 16
None 17
None 18
None 19
None 20
None 21
None 22
None 23
None 24
None 25
None 26
None 27
None 28
None 29
None 30
None 31
None 32
None 33
None 34
None 35
None 36
None 37
None 38
None 39
None 40
None 41
None 42
None 43
None 44
None 45
None 46
None 47
None 48
None 49
None 50
None 51
None 52
None 53
None 54
None 55
None 56
None 57
None 58
None 59
None 60
None 61
None 62
None 63
None 64
None 65
None 66
None 67
None 68
None 69
None 70
None 71
None 72
None 73
None 74
None 75
None 76
None 77
None 78
None 79
None 80
None 81
None 82
None 83
None 84
None 85
None 86
None 87
None 88
None 89
None 90
None 91
None 92
None 93
None 94
None 95
None 96
None 97
None 98
None 99
None 100
None 101
None 102
None 103
None 104
None 105
None 106
None 107
None 108
None 109
None 110
None 111
None 112
None 113
None 114
None 115
None 116
None 117
None 118
None 119
None 120
None 121
None 122
None 123
N

None 926


KeyboardInterrupt: 

In [10]:
def get_corrs(pc):
    
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(pc))
      lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude, longitude

In [12]:
post_code='M7A'
lat, long = get_corrs(post_code)

KeyboardInterrupt: 